In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils
import utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_filter_map_plot

E0415 22:14:21.533650042    1355 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0415 22:14:24.121443542    1355 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
## parameters cell
itp_id = 290

In [3]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    >> filter(_.itp_id == itp_id)
    >> collect()
)

In [4]:
# airtable_organizations

In [5]:
analysis_date = dt.date(2022, 3, 30)

In [6]:
trips = pd.read_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{itp_id}_03_30.parquet')

In [7]:
stop_delay = gpd.read_parquet(f'{utils.GCS_FILE_PATH}stop_delay_views/{itp_id}_03_30.parquet')
stop_delay['arrival_time'] = stop_delay.arrival_time.map(lambda x: dt.datetime.fromisoformat(x))
stop_delay['actual_time'] = stop_delay.actual_time.map(lambda x: dt.datetime.fromisoformat(x))

In [8]:
%%capture
routelines = utils.get_routelines(itp_id, analysis_date)

In [9]:
%%capture
rt_day = rt_filter_map_plot.RtFilterMapper(trips, stop_delay, routelines)

In [10]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')

In [11]:
rt_day.describe_delayed_routes()

PM Peak most delayed routes: 
 * 112, Linda Mar Park & Ride - Colma BART, Southbound: 24.0 minutes late on average
 * 17, Linda Mar Park & Ride - Pescadero, Northbound: 19.0 minutes late on average
 * 18, Main/7th - HMBHS - Moonridge Apts, Northbound: 14.0 minutes late on average
 * 28, Serramonte Center - South City High, Westbound: 13.0 minutes late on average
 * 110, Daly City BART - Linda Mar Park & Ride, Southbound: 10.0 minutes late on average

In [2]:
import gcsfs
fs = gcsfs.GCSFileSystem()

In [3]:
fs_list = fs.ls(f'{utils.GCS_FILE_PATH}rt_trips/')

In [4]:
ran_operators = [int(path.split('rt_trips/')[1].split('_')[0])
                 for path in fs_list
                 if path.split('rt_trips/')[1]]

In [5]:
ran_operators

[110,
 126,
 127,
 135,
 148,
 159,
 167,
 170,
 182,
 188,
 194,
 208,
 218,
 221,
 226,
 235,
 243,
 246,
 259,
 269,
 273,
 278,
 282,
 284,
 290,
 293,
 294,
 295,
 300,
 301,
 30,
 310,
 315,
 336,
 349,
 350,
 361,
 45,
 4,
 75]

In [6]:
import os

In [7]:
nb_exists = os.listdir('../docs/rt_delay/')

In [8]:
nb_exists = [int(nb.split('id_')[1].split('.')[0]) for nb in nb_exists if 'id_' in nb]

In [9]:
[operator for operator in ran_operators if operator not in nb_exists]

[273]